# <div align="center"><b> ОПРЕДЕЛЕНИЕ СТОИМОСТИ АВТОМОБИЛЕЙ </b></div> 

**Цель:** создать модель предсказания рыночной стоимости автомобиля;

**Заказчик:** сервис по продаже автомобилей с пробегом "Не бит, не крашен";

**Исходные данные:** набор данных о технических характеристиках, комплектации и ценах других автомобилей;

**Критерии:** качество предсказания (значение метрики RMSE), время обучения модели, время предсказания модели

# 1. Подготовка данных

## 1.1 Загрузка и обзор данных

In [ ]:
#установим библиотеку для автоматического EDA
!pip install pandas_profiling

In [ ]:
#установим библиотеку LightGBM
!pip install lightgbm

In [ ]:
#импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import pandas_profiling
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
#снимем ограничение на количество столбцов датафрейма, выводимых на экран
pd.options.display.max_columns = None

In [ ]:
#сформируем датафрейм
try:
    #берем данные с локального компьютера
    df = pd.read_csv(r'C:\Users\vizum\Desktop\yandex_practicum\project_11\datasets\autos.csv')
except:
    #берем данные с хранилища Яндекс.Практикума
    df=pd.read_csv('/datasets/autos.csv')
df

In [ ]:
#для быстрого анализа данных воспользуемся библиотекой pandas_profiling
#pandas_profiling.ProfileReport(df)

Анализируя статистическую информацию о данных, можно сделать следующие выводы:
- названия столбцов приведены в стиле CamelCase, перименовывать не будем;
- в датафрейме 4 строки-дубликата - их удалим;
- в столце Price 3% нулевых значений. Строки с этими значениями далее удалим, т.к. они бесполезны для обучения;
- необходимо проверить столбцы 'VehicleType', 'Model', 'FuelType', 'Brand' на наличие неявных дубликатов, т.к. данных, предоставленных в отчете недостаточно;
- в столбце с годом регистрации автомобиля присутствуют аномальные значения: год регистрации варьируется от 1000 до 9999, что невозможно;
- мощность авто варьируется от 0 до 20000 л.с., что также невозможно, поскольку на текущий момент максимальная мощность авто составляет 5000 л.с, согласно [источнику](https://cars-rating.ru/legkovye/samye-moshhnye-avtomobili-v-mire?ysclid=l87umrf7dy554612692). Необходимо подробнее рассмотреть данный параметр;
- в приведенных объявлениях полностью отсутствует информация о фотографиях авто - все значения нулевые;
- максимальное количество пропусков (20%) наблюдается в столбце 'NotRepaired', содержащего информацию о факте проведения ремонтных работ с продаваемым автомобилем;
- теоретически, по столбцу с почтовым индексом можно было бы определить страну продажи авто, однако данных для идентефикации страны недостаточно - один почтовый индекс может принадлежать городам в разных странах;
- в столбце с месяцем регистрации должно быть 12 уникальных значений, однако присутствует 13 - от 0 до 12 включительно;
- столбцы 'DateCrowled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen' следует удалить, так как они не нужны для обучения модели.

## 1.2 Предобработка данных

In [ ]:
#сохраним начальную длину датафрейма
df_length_start = df.shape[0]

In [ ]:
#удалим строки с нулевой ценой
df = df[df['Price']!=0]
print(df.shape)
print('Удалено от первоначального датафрейма, %:', (df_length_start/df.shape[0]-1)*100)

In [ ]:
#построим гистограмму распределения цены
df['Price'].plot(kind='hist', figsize=(19,10), bins=200, ylabel = 'Цена авто, евро', xlabel='Индекс строки', title='Точечный график стоимости автомобилей в датасете');
plt.minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', linestyle=':')

In [ ]:
#построим точечный график распределения цены
df['Price'].plot(style='o', figsize=(19,10), ylim=(0, 500), ylabel = 'Цена авто, евро', xlabel='Индекс строки', title='Точечный график стоимости автомобилей в датасете');
plt.minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', linestyle=':')

In [ ]:
#построим гистограмму распределения цены для цен до 500 евро
df[df['Price']<500]['Price'].plot(kind='hist', figsize=(19,10), bins=200, ylabel = 'Цена авто, евро', xlabel='Индекс строки', title='Точечный график стоимости автомобилей в датасете');
plt.minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', linestyle=':')

In [ ]:
#посмотрим на строки датасета с небольшими ценами
df[df['Price']<10]

In [ ]:
df = df[df['Price']>50]
df['Price'].min()

In [ ]:
#удалим строки с нулевой ценой
print('Удалено от первоначального датафрейма, %:', (df_length_start/df.shape[0]-1)*100)

In [ ]:
#удалим полные дубликаты
df=df.drop_duplicates()
df.duplicated().sum()

In [ ]:
#проверим столбцы 'VehicleType', 'Model', 'FuelType', 'Brand' на наличие неявных дубликатов
for column in ['VehicleType', 'Model', 'FuelType', 'Brand']:
    print('='*30, column, '='*30)
    print(df[column].sort_values().unique(), '\n')

Найдены неявные дубликаты в столбце 'Model': rangerover и range_rover.

In [ ]:
#избавимся от неявных дубликатов в столбце с моделью авто
df['Model']=df['Model'].replace(['rangerover'], 'range_rover')
df['Model'].sort_values().unique()

**Рассмотрим аномальные значения в столбце с годом регистрации автомобиля.**

In [ ]:
#для быстрого обзора ситуации построим точечный график
df['RegistrationYear'].plot(style='o', figsize=(19, 10), ylabel = 'Год регистрации', xlabel='Индекс строки', title='Значения года регистрации автомобилей');
plt.minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', linestyle=':')

Анализируя график выше, можно сказать, что аномальных значений не очень много, основная часть значений сконцентрирована в районе 2000-х годов. Увеличим масштаб по оси Y:

In [ ]:
#построим точечный график значений года регистрации с увеличенным масштабом по оси Y
df['RegistrationYear'].plot(style='o', figsize=(19, 10), ylim=(1000, 2022), ylabel = 'Год регистрации', xlabel='Индекс строки', title='Значения года регистрации автомобилей');
plt.minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', linestyle=':')

Явно аномальные значения года регистрации расположены до 1800 года включительно. Небольшая скученность значений наблюдается в пределах от 1900 до 1950 года. Посмотрим на строки датасета в этом промежутке.

In [ ]:
df[(df['RegistrationYear']<1950)&(df['RegistrationYear']>1900)].sort_values(by='RegistrationYear')

Анализируя фрагмент датасета выше, можно выявить временные несостыковки:
- первая машина марки trabant, согласно [Википедии](https://translated.turbopages.org/proxy_u/en-ru.ru.6b369785-6328b591-0a00d8ef-74722d776562/https/en.wikipedia.org/wiki/Trebant) была выпущена в 1957 году, в то время как в датасете указан год регистрации автомобиля 1910. 
- первая машина компании марки volkswagen была выпущена в [1938 году](https://rus.team/events/vypuschen-pervyy-avtomobil-folksvagen?ysclid=l893y3nr0451568959), однако в датасете присутствует автомобиль этой марки от 1910 года.

В то же время наблюдаются совпадения для более поздних годов:
- в 1949-1954 годах [выпускался](https://automotive-heritage.com/model/787?ysclid=l894j9jh5u58108009) автомобиль Chrysler Imperial Sedan - и в датасете под индексом 27525 присутствует автомобиль Chrysler Sedan с 1949 годом регистрации;
- в 1947 году была [выпущена](https://automotive-heritage.com/model/829?ysclid=l894rytlp5884452730) модель Chrysler Town & Country Sedan (1947) - в датасете под индексом 326241 присутствует автомобиль Chrysler Sedan с 1947 годом регистрации

Рассмотрим временной отрезок с 1900 по 1950 гг на графике в укрупненном масштабе.

In [ ]:
#построим точечный график значений года регистрации в промежутке от 1900 до 1950 гг
df['RegistrationYear'].plot(style='o', figsize=(19, 10), ylim=(1900, 1950), ylabel = 'Год регистрации', xlabel='Индекс строки', title='Значения года регистрации автомобилей');
plt.minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', linestyle=':')

Действительно, график значений года регистрации в промежутке от 1900 до 1950 гг имеет аномальную "прямую" Y=1910. Вероятнее всего, корректные значения находятся в промежутке от 1920 по 2022 гг. Рассчитаем процент строк, не входящих в этот промежуток.

In [ ]:
df[(df['RegistrationYear']>2022)|(df['RegistrationYear']<1920)]['Price'].count()*100/df.shape[0]

Процент аномальных значений в 'RegistrationYear' довольно мал. Удалим строки с аномальными значениями.

In [ ]:
print('Количество строк до удаления аномальных значений в registrationYear:', df.shape[0])
df = df[(df['RegistrationYear']<=2016)&(df['RegistrationYear']>1920)]
print('Количество строк после удаления аномальных значений в registrationYear:', df.shape[0])


In [ ]:
print('Удалено от первоначального датафрейма, %:', (df_length_start/df.shape[0]-1)*100)

**Рассмотрим аномальные значения в столбце с мощностью автомобиля.**

In [ ]:
#для быстрого обзора ситуации построим точечный график
df['Power'].plot(style='o', figsize=(19, 10), ylabel = 'Мощность, л.с.', xlabel='Индекс строки', title='График значений мощности автомобилей');
plt.minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', linestyle=':')

Согласно графику выше, основная часть значений мощности все-таки находится в пределах от 0 до 2500 л.с. Рассмотрим аномально большие и аномально маленькие значения.

In [ ]:
df[df['Power']>2700].sort_values(by='Power', ascending=False)

Мощность mercedes_benz clk 1999 года, согласно датасету, составляет 20000 л.с. Максимальная мощность автомобиля CLK 320 AT Elegance, выпускавшегося с 1998 по 2000 гг, составляет [218 л.с.](https://www.drom.ru/catalog/mercedes-benz/clk-class/231415/?ysclid=l8a7c3j30481566423). Можно предположить, что аномальные значения больше реальных в 10 или 100 раз - в таком случае мощность автомобиля bmw 1er sedan 2006 г (114106 строка датасета) должна составлять 192.1 л.с. Однако автомобили такой модели с таким типом кузова с мощностью 192.1 л.с. [отсутствуют](https://www.drom.ru/catalog/bmw/1-series/2006/?ysclid=l8a7uwnhfa196023455). Определим процент значений выше 1000 л.с.


In [ ]:
df[df['Power']>1000]['Price'].count()/df.shape[0]*100

Процент значений свыше 1000 л.с. очень мал. Удалим эти значения.

In [ ]:
df = df[df['Power']<=1000]
df['Power'].max()

In [ ]:
print('Удалено от первоначального датафрейма, %:', (df_length_start/df.shape[0]-1)*100)

Построим гистограмму распределения значений мощности на 100 корзин.

In [ ]:
df['Power'].plot(kind='hist', figsize=(19,10), bins=200, ylabel = 'Количество случаев', xlabel='Мощность л.с.', title='Гистограмма распределения значений мощности автомобилей')
plt.minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', linestyle=':')

На гистограмме распределения значений мощности автомобиля видим локальный аномальный пик возле 0 (более 36 тысяч объявлений). Удалим "хвост" со значениями можности более 400 и нулевые значения, снова построим гистограмму распределения.

In [ ]:
#рассчитаем долю нулевых значений мощности
df[df['Power']==0]['Price'].count()/df.shape[0]

Доля нулевых значений очень большая - 10%, тысячи строк. Эти данные не восстановить, точно удаляем :(

In [ ]:
df = df[(df['Power']<=400)&(df['Power']!=0)]
print(df['Power'].max())
print(df['Power'].min())

In [ ]:
print('Удалено от первоначального датафрейма, %:', (df_length_start/df.shape[0]-1)*100)

In [ ]:
df['Power'].plot(kind='hist', figsize=(19,10), bins=200, ylabel = 'Количество случаев', xlabel='Мощность л.с.', title='Гистограмма распределения значений мощности автомобилей')
plt.minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', linestyle=':')

In [ ]:
#посмотрим на модели с мощностью 40 л.с. и выше
df[df['Power']>=40].sort_values(by='Power').head(30)

Для некоторых моделей (opel corsa, smart fortwo) значения соответствуют указанным. Удалим строки со значениями мощности меньше 40.

In [ ]:
df = df[df['Power']>=40]
df['Power'].describe()

In [ ]:
print('Удалено от первоначального датафрейма, %:', (df_length_start/df.shape[0]-1)*100)

**Рассмотрим аномальные значения в столбце c месяцем регистрации**

In [ ]:
df['RegistrationMonth'].value_counts().reset_index().plot(kind='bar', ylabel = 'Количество случаев', xlabel='Значение месяца регистрации', title='Количество объявлений на каждый месяц регистрации авто', figsize=(19, 10));
plt.minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', linestyle=':')

Без дополнительной информации аномальное количество месяцев не исправить. Столбец удалим далее.

**Заполнение пропусков**

In [ ]:
#получим общую информацию о пропусках в df
df.info()

Пропуски присутствуют только в столбцах с категориальными данными. Заполним все пропуски значением 'unknown'.

In [ ]:
df=df.fillna('unknown')

In [ ]:
#проверим результат
df.info()

Пропусков нет.

**Удаление неинформативных столбцов**

Удалим столбцы 'DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen' как не информативные для обучения, а также столбец 'RegistrationMonth', содержащий недостоверные данные.

In [ ]:
#удалим лишние столбцы
df = df.drop(columns=['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen', 'RegistrationMonth'])

In [ ]:
#проверим результат
df.columns

## 1.3 Подготовка выборок для обучения

- выделим основные и целевые признаки в обучающей и тестовой выборках;
- выполним прямое кодирование категориальных признаков;
- разделим выборки на обучающую и тестовую в соотношении 3:1. Валидационную выборку отдельно выделять не будем, так как будем использовать кроссвалидацию.

In [ ]:
#разделим закодированный датасет на признаки и целевой признак
features = df.drop(columns=['Price']).reset_index(drop=True)
target = df[['Price']].reset_index(drop=True)

print(features.columns)
print(target.columns)

In [ ]:
features_ohe = features
features_ohe

In [ ]:
#выполним прямое кодирование категориальных признаков для модели линейной регрессии
features_ohe = pd.get_dummies(features_ohe, columns=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'], drop_first=True)
features_ohe

In [ ]:
#выполним порядковое кодирование категориальных признаков для градиентного бустинга и дерева решений
categorical = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
encoder = OrdinalEncoder()
features[categorical] = pd.DataFrame(encoder.fit_transform(features[categorical]), columns=categorical)
features

In [ ]:
#разделим выборки 
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)
features_train_ohe, features_test_ohe, target_train_ohe, target_test_ohe = train_test_split(features_ohe, target, test_size=0.25, random_state=12345)

In [ ]:
#проверим размерности выборок для бустинга и ДР
print('features_train:', features_train.shape)
print('features_test:', features_test.shape)
print('target_train:', target_train.shape)
print('target_test:', target_test.shape)

In [ ]:
#проверим размерности выборок для бустинга и ДР
print('features_train_ohe:', features_train_ohe.shape)
print('features_test_ohe:', features_test_ohe.shape)
print('target_train_ohe:', target_train_ohe.shape)
print('target_test_ohe:', target_test_ohe.shape)

Выполним стандартизацию данных.

In [ ]:
numeric = ['RegistrationYear', 'Power', 'Kilometer']

In [ ]:
#создадим и настроим StandardScaler
scaler = StandardScaler()
scaler.fit(features_train[numeric])

#стандартизируем признаки в features_train и features_valid
features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
features_train_ohe[numeric] = scaler.transform(features_train_ohe[numeric])
features_test_ohe[numeric] = scaler.transform(features_test_ohe[numeric])

# 2. Обучение моделей

## 2.1 Обучение модели с помощью градиентного бустинга (библиотека LightGBM)

In [ ]:
#сформируем список с набором гиперпараметров
parameters = {'n_estimators':[50, 100, 200, 300, 400], 'max_depth':[1, 2, 3, 4, 5, 6, 7], 'reg_lambda':[0, 0.3, 0.7]}

In [ ]:
%%time
#подберем наилучшие гиперпараметры модели с помощью градиентного бустинга
model_lgbm= GridSearchCV(LGBMRegressor(random_state=12345), param_grid=parameters, scoring = 'neg_root_mean_squared_error', cv=5, verbose=1)
model_lgbm.fit(features_train, target_train)

In [ ]:
print('Наилучшие параметры модели:', model_lgbm.best_params_)

#сохраним оценку модели, полученную с помощью кроссвалидации
best_score_lgbm = model_lgbm.best_score_
print('Оценка модели решений:', best_score_lgbm)

In [ ]:
%%time
#оценим время обучения модели
model_lgbm_best = LGBMRegressor(random_state=12345, **model_lgbm.best_params_)
model_lgbm_best.fit(features_train, target_train)

In [ ]:
%%time
#оценим время предсказания и выполним предсказание модели по тестовым данным
predict = model_lgbm_best.predict(features_train)

In [ ]:
%%time
#оценим время предсказания и выполним предсказание модели по тестовым данным
#predict = model_lgbm_best.predict(features_test)

In [ ]:
#рассчитаем точность предсказания
#rmse_lgbm = (mean_squared_error(predict, target_test))**0.5
#rmse_lgbm

Сохраним результаты в словарь:

In [ ]:
results = {'Model':['model_lgbm_best', 'model_lr', 'model_dt_best'], 'validation_time, m':[], 'training_time, s':[], 'pred_time, s':[], 'val_rmse':[], 'best_params':[]}
results

In [ ]:
results['validation_time, m'].append(round(8+6/60, 2))
results['training_time, s'].append(1.7)
results['pred_time, s'].append(0.99)
results['val_rmse'].append(round(-best_score_lgbm, 2))
results['best_params'].append(model_lgbm.best_params_)

results

## 2.2 Обучение модели с помощью линейной регрессии

In [ ]:
#зададим алгоритм модели
model_lr = LinearRegression()

In [ ]:
%%time
#получим вектор оценок качества модели по результатам кросс-валидации
scores_lr = cross_val_score(model_lr, features_train_ohe, target_train_ohe, cv=5, scoring='neg_root_mean_squared_error')
print(scores_lr)

In [ ]:
print('Средняя оценка:', pd.Series(scores_lr).mean())

In [ ]:
%%time
#обучим модель на тренировочных данных
model_lr.fit(features_train, target_train)

In [ ]:
%%time
#рассчитаем время предсказания
#predict_lr = model_lr.predict(features_test)

In [ ]:
%%time
#рассчитаем время предсказания
predict_lr = model_lr.predict(features_train)

In [ ]:
#оценим качество предсказания
#rmse_lr = (mean_squared_error(predict_lr, target_test))**0.5
#rmse_lr

In [ ]:
#сохраним результаты
results['validation_time, m'].append(round(15/60, 2))
results['training_time, s'].append(0.05)
results['pred_time, s'].append(0.007)
results['val_rmse'].append(round(-pd.Series(scores_lr).mean(), 2))
results['best_params'].append('default')

results

## 2.3 Обучение модели с помощью дерева решений

In [ ]:
#сформируем список с набором гиперпараметров
parameters_dt = {'min_samples_split':[2, 3, 4, 5, 6, 7, 8, 9, 10], 'max_depth':[3, 4, 5, 6, 7, 8, 9, 10]}

In [ ]:
%%time
model_dt= GridSearchCV(DecisionTreeRegressor(random_state=12345), param_grid=parameters_dt, scoring = 'neg_root_mean_squared_error', cv=5, verbose=1)
model_dt.fit(features_train, target_train)

In [ ]:
print('Наилучшие параметры модели:', model_dt.best_params_)

#сохраним оценку модели, полученную с помощью кроссвалидации
best_score_dt = model_dt.best_score_
print('Оценка модели на основе дерева решений:', best_score_dt)

In [ ]:
%%time
#оценим время обучения модели
model_dt_best = DecisionTreeRegressor(random_state=12345, **model_dt.best_params_)
model_dt_best.fit(features_train, target_train)

In [ ]:
%%time
#рассчитаем время предсказания
#predict_dt = model_dt_best.predict(features_test)

In [ ]:
%%time
#рассчитаем время предсказания
predict_dt = model_dt_best.predict(features_train)

In [ ]:
#оценим качество предсказания
#rmse_dt = (mean_squared_error(predict_dt, target_test))**0.5
#rmse_dt

In [ ]:
#сохраним результаты
results['validation_time, m'].append(round(1+27/60, 2))
results['training_time, s'].append(0.42)
results['pred_time, s'].append(0.03)
results['val_rmse'].append(round(-best_score_dt, 2))
results['best_params'].append(model_dt.best_params_)

results

# 3. Анализ построенных моделей

Сформируем табличку с характеристиками построенных моделей.

In [ ]:
pd.DataFrame(results)

- **Model** - имя модели предсказания стоимости автомобиля;
    - ***model_lgbm_best*** - модель на базе случайного леса, обученная градиентным бустингом;
    - ***model_lr*** - модель на базе алгоритма линейной регрессии;
    - ***model_dt_best*** - модель на базе алгоритма дерева решений;
- **validation_time** - время подбора гиперпараметров модели с помощью кроссвалидации при разбиении обучающей выборки на 5 частей. Для линейной регрессии подбор гиперпараметров не осуществлялся;
- **training_time** - время обучения модели с наилучшими гиперпараметрами;
- **pred_time** - время предсказания модели;
- **val_rmse** - качество модели на кроссвалидации;
- **test_rmse** - качество предсказания;
- **best_params** - наилучшие гиперпараметры.

Анализируя сводную таблицу характеристик моделей прогноза стоимости автомобиля, можно сказать следующее: 
- требованиям точности к модели отвечают две модели: model_lgbm_best и model_dt_best - значения RMSE этих моделей на валидации лучше порогового (2500). Однако наилучшее качество модели, по метрике RMSE, наблюдается для модели model_lgbm_best и составляет 1568. Полученное значение лучше RMSE моделей случайного леса и линейной регрессии в 1.2 и 1.8 раз соответственно. Модель model_lr не проходит по требованиям точности: RMSE линейной регрессии на валидации составила 2х10^12, что много больше допустимого значения 2500. Далее модель model_lr рассматривать не будем;
- по времени обучения модель model_dt_best быстрее модели model_lgbm_best в 4 раза, по времени предсказания - в 33 раза. Однако с учетом объема тренировочной выборки в 220.5 тысяч строк время предсказания обеих моделей можно признать небольшим.

Модель model_dt_best опережает модель model_lgbm_best по двум критериям, связанным со скоростью работы, однако проигрывает по точности предсказания. Так как точность - единственный регламентируемый параметр, примем его более важным, чем скорость работы.

С учетом вышесказанного, в качестве **итоговой** можно принять модель model_lgbm_best.

В качестве **запасной** модели можно принять model_dt_best - для случая, когда скорость предсказания будет более критична, чем точность.

**Проверим точность итоговой и запасной моделей на тестовой выборке**

In [ ]:
#предсказание для итоговой модели
test_predict_lgbm = model_lgbm_best.predict(features_test)
rmse_lgbm = (mean_squared_error(test_predict_lgbm, target_test))**0.5
rmse_lgbm

In [ ]:
#предсказание для запасной модели
test_predict_dt = model_dt_best.predict(features_test)
rmse_dt = (mean_squared_error(test_predict_dt, target_test))**0.5
rmse_dt

**Сравним выбранные модели с константной**

In [ ]:
#создадим предсказание константной модели
const_model = np.array([target_train.median()]*target_test.shape[0])
const_model

In [ ]:
#сравним итоговую модель с константной
rmse_const_f = (mean_squared_error(test_predict_lgbm, const_model))**0.5
rmse_const_f

In [ ]:
#сравним итоговую модель с константной
rmse_const_z = (mean_squared_error(test_predict_dt, const_model))**0.5
rmse_const_z

Точность константной модели хуже итоговой и запасной моделей в три и два раза соответственно. Выбранные модели вменяемы.

# 4. Общий вывод

В рамках создания модели предсказания рыночной стоимости автомобиля для компании "Не бит, не крашен" выполнено следующее:
- выполнена предобработка данных, в результате которой было удалено 16% значений. Такой процент удаленных данных вызван большим количеством аномальных (нулевых) значений в параметре 'Power', которые было невозможно восстановить;
- выполнено обучение трех моделей:
    - на основе случайного леса способом градиентного бустинга LightGBM (model_lgbm_best);
    - на основе линейной регрессии (model_lr);
    - на основе дерева решений (model_dt_best).

- наилучшую точность на валидации показала модель model_lgbm_best: RMSE=1568;
- наиболее быстрая модель, отвечающая требованиям к точности: model_lgbm_best, RMSE на валидации 1947.
- модель model_lr не удовлетворяет требованиям точности. RMSE на валидации составляет 2567, что превышает допустимое значение, равное 2500;

Из предположения, что точность предсказания является приоритетным критерием, в качестве итоговой была выбрана модель **model_lgbm_best** с параметрами {'max_depth': 7, 'n_estimators': 400, 'reg_lambda': 0.3}. Точность этой модели на тестовых данных составила RMSE = 1562.
В случае, когда скорость предсказания более критична, чем точность, можно использовать модель **model_dt_best** с параметрами {'max_depth': 10, 'min_samples_split': 10}. Точность этой модели на тестовых данных составила RMSE = 1930.

In [ ]:
model_lgbm_best = LGBMRegressor(random_state=12345)
model_lgbm_best.fit(features_train, target_train)

In [ ]:
pd.Series(model_lgbm_best.feature_importances_, index=features_train.columns).sort_values().plot(kind='bar', ylabel = 'Важность признака', xlabel='Признак', title='Важность признаков для модели LightGBM', figsize=(19, 10));
plt.minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', linestyle=':')

Для итоговой модели наиболее важные признаки: возраст машины и мощность двигателя. На втором месте находятся модель и марка автомобиля, на третьем - тип кузова и пробег. 
Наименьшую роль в ценообразовании играет тип коропки передач, наличие факта ремонта и тип топлива.